In [1]:
import mbuild as mb
import numpy as np

In [2]:
class CH2(mb.Compound):
    """A methylene bridge. """
    def __init__(self):
        super(CH2, self).__init__()
        
        mb.load('pdb-files/ch2.pdb', compound=self)
        self.add(mb.Port(anchor=self[0], orientation=[0, 1, 0], separation=0.075), label='up')
        self.add(mb.Port(anchor=self[0], orientation=[0, -1, 0], separation=0.075), label='down')
        
class H(mb.Compound):
    """A hydrogen atom. """
    def __init__(self):
        super(H, self).__init__()
        
        self.add(mb.Particle(name='H'))
        self.add(mb.Port(anchor=self[0], orientation=[0, 1, 0], separation=0.075), label='up')
        
class Alkane(mb.Compound):
    """A linear alkane chain. """
    def __init__(self, n):
        """Initialize an Alkane Compound.

        Parameters
        ----------
        n : int
            Number of carbon atoms in the chain
            
        """
        if n < 1:
            raise ValueError('n must be 1 or more')
        super(Alkane, self).__init__()

        chain = mb.Polymer(CH2(), n=n, port_labels=('up', 'down'))
        self.add(chain, 'chain')
        
        up_cap = H()
        down_cap = H()
        mb.force_overlap(up_cap, up_cap['up'], chain['up'])
        self.add(up_cap)
        mb.force_overlap(down_cap, down_cap['up'], chain['down'])
        self.add(down_cap)
        
class AlkaneBox(mb.Compound):
    """A box of alkane chains. """
    def __init__(self, n, n_chains, box_length):
        super(AlkaneBox, self).__init__()
        
        chain = Alkane(n)
        box = mb.Box(lengths=[box_length, box_length, box_length])
        box_of_chains = mb.fill_box(compound=chain, n_compounds=n_chains, box=box)
        self.add(box_of_chains)

In [3]:
system = AlkaneBox(n=5, n_chains=200, box_length=4.0)
system.save('system.gro', overwrite=True)
system.save('system.top', forcefield_name='oplsaa', overwrite=True)

/Users/summeraz/foyer/foyer/validator.py:163: ValidationWarning: There are 703 atom types that are missing a smarts definition. To view the missing atom types, re-run with debug=True when applying the forcefield.
  "applying the forcefield.".format(len(missing_smarts)), ValidationWarning)


In [ ]:
gmx grompp -f mdp_files/em.mdp -c system.gro -p system.top -o em.tpr
gmx mdrun -v -deffnm em

In [ ]:
gmx grompp -f mdp_files/equil.mdp -c em.gro -p system.top -o equil.tpr
gmx mdrun -v -deffnm equil

In [ ]:
gmx grompp -f mdp_files/sample.mdp -c equil.gro -p system.top -o sample.tpr
gmx mdrun -v -deffnm sample